In [48]:
# There are  missing values in our data set lets replace them with the mean. 
X = X.fillna(X.mean())

print(len(X[X.isna().any(axis=1)]))
print(len(y[y.isna().any(axis=1)])) # we also make sure there's no missing value for y . 

0
0


In [32]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import feature_selection
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

#Outlier detection
def filter(df1, df2):
  # Filter feature selection
  from sklearn.feature_selection import VarianceThreshold 
  
  # Step 1: Removing Constant features
  constant_filter = VarianceThreshold(threshold=0)
  data_constant = constant_filter.fit_transform(df1)
  #print(data_constant.shape)
  constant_columns = [column for column in df1.columns if column not in df1.columns[constant_filter.get_support()]]
  data_cons1 = df1.drop(constant_columns,axis=1)
  data_cons2 = df2.drop(constant_columns,axis=1)
      
  # Step 2: Removing Quasi-Constant Features
  qcons_filter = VarianceThreshold(threshold=0.001)
  data_qcons = qcons_filter.fit_transform(df1)
  #print(data_qcons.shape)
  qcons_columns = [column for column in df1.columns if column not in df1.columns[qcons_filter.get_support()]]
  data_qcons1 = df1.drop(qcons_columns,axis=1)
  data_qcons2 = df2.drop(qcons_columns,axis=1)
  data_qcons_t1 = data_qcons1.T
  data_qcons_t2 = data_qcons2.T 
  
  # Step 3: Removing Duplicate Columns
  data_cons_dup1 = data_qcons_t1.drop_duplicates(keep='first').T
  data_cons_dup2 = data_qcons_t2.drop_duplicates(keep='first').T

  return data_cons_dup1, data_cons_dup2

def reduce_feats(X):
    ### performs PCA on the preprocessed features ###
    pca = PCA(n_components=0.75, whiten=True)
    return pca.fit_transform(X)



X_test = pd.read_csv("X_test.csv", index_col=0).values
X_train = pd.read_csv("X_train.csv", index_col=0).values
y_train = pd.read_csv("y_train.csv", index_col=0).values

df_train = pd.DataFrame(X_train)
df_test = pd.DataFrame(X_test)

# There are  missing values in our data set lets replace them with the mean. 
df_train = df_train.fillna(df_train.mean())
df_test = df_test.fillna(df_test.mean())

X_train, X_test = filter(df_train, df_test)
X_train, X_test = X_train.values, X_test.values

X_scaler = StandardScaler()
y_scaler = StandardScaler()

# Apply std scaler and reduce dimensions 
X_train_scaled = X_scaler.fit_transform(X_train)
X_train = reduce_feats(X_train_scaled )
#y_train_scaled = y_scaler.fit_transform(y_train)

print(X_train.shape)

# Split the data into training set and testing set
X_train_model, X_val, y_train_model, y_val = train_test_split(X_train, y_train, 
                                                              test_size = 0.2, random_state = 42)




(1212, 252)


In [29]:
print(X_train.shape)

(1212, 773)
